# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [61]:
# Write your code below.
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [62]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [63]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet/part.*.parquet"))

In [65]:
parquet_files

['../../05_src/data/prices\\A\\A_2000.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2001.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2002.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2003.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2004.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2005.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2006.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2007.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2008.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2009.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2010.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2011.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2012.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2013.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2014.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2015.pa

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [66]:
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

dd_feat = (
    dd_px.groupby("ticker", group_keys=False)
        .apply(
                lambda x: x.assign(
                    Close_lag = x["Close"].shift(1),
                    Adj_Close_lag = x["Adj Close"].shift(1))
            ).assign(
                returns = lambda x: x["Adj Close"]/x["Adj_Close_lag"] - 1
            ).assign(
                hi_low_range = lambda x: x["High"] - x["Low"]
            ))




In [67]:
# Confirm which columns are available in the Dask data frame

import pandas as pd

sample_file = parquet_files[0]
df_sample = pd.read_parquet(sample_file)
print(df_sample.columns)


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sector',
       'subsector', 'year'],
      dtype='object')


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Convert Dask data frame into pandas data frame

pd_feat = dd_feat.compute()

# Calculate a 10 day rolling average return
pd_feat = (
    pd_feat.groupby("ticker", group_keys=False)
    .apply(lambda x: x.assign(rolling_avg_returns=x["returns"].rolling(10).mean()))
)




In [75]:
pd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_low_range,rolling_avg_returns
ticker,,,,,,,,,,,,,,,
HUM,2015-01-02,144.949997,146.509995,141.389999,142.990005,133.775009,982800,Health Care,Managed Health Care,2015,NaN,NaN,NaN,5.119995,NaN
HUM,2015-01-05,141.690002,142.759995,138.690002,139.199997,130.229324,1686700,Health Care,Managed Health Care,2015,142.990005,133.775009,-0.026505,4.069992,NaN
HUM,2015-01-06,140.210007,141.490005,137.449997,139.089996,130.126328,1770100,Health Care,Managed Health Care,2015,139.199997,130.229324,-0.000791,4.040009,NaN
HUM,2015-01-07,139.729996,142.240005,139.729996,141.729996,132.596222,877400,Health Care,Managed Health Care,2015,139.089996,130.126328,0.018981,2.510010,NaN
HUM,2015-01-08,143.619995,148.389999,143.619995,147.830002,138.303070,1350000,Health Care,Managed Health Care,2015,141.729996,132.596222,0.043039,4.770004,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JNPR,2015-12-24,27.620001,27.780001,27.559999,27.620001,22.041357,1511300,Information Technology,Communications Equipment,2015,27.650000,22.065300,-0.001085,0.220001,-0.004224
JNPR,2015-12-28,27.580000,27.750000,27.430000,27.709999,22.113180,2804200,Information Technology,Communications Equipment,2015,27.620001,22.041357,0.003259,0.320000,-0.003066
JNPR,2015-12-29,27.870001,28.230000,27.850000,28.040001,22.376522,2783100,Information Technology,Communications Equipment,2015,27.709999,22.113180,0.011909,0.379999,-0.002399


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Yes, it was necessary to convert to pandas to calculate the moving average return because it required accessing sequential data points across rolls.

It would have NOT been better to do it in Dask because Dask processes data in groups which is complicated when the rolling calculations span multiple partitions. Panas is more efficient for complex calculations and when the dataset fits in memory.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.